In [ ]:
#pip install azure-eventhub

In [ ]:
#pip install azure-eventhub-checkpointstoreblob-aio

# Code below works to send message to eventhub.  Eventhub sends batch of messages to blobstorage

In [32]:
from azure.eventhub import EventHubProducerClient, EventData

eventhub_name = 'eventhub'
client = EventHubProducerClient.from_connection_string(os.environ.get('CONNECTION_STRING'), eventhub_name=eventhub_name)

event_data_batch = client.create_batch()
can_add = True
while can_add:
    try:
        event_data_batch.add(EventData('{"messageId":3,"deviceId":"Python Client","temperature":43.22222,"humidity":22.11111}'))  #JSON string
    except ValueError:
        can_add = False  # EventDataBatch object reaches max_size.

with client:
    client.send_batch(event_data_batch)

In [13]:
import os

os.environ['CONNECTION_STRING'] = 'Endpoint=sb://pythoneventhb.servicebus.windows.net/;SharedAccessKeyName=eventpolicy;SharedAccessKey=gR5uUifVoEXQpY57GvQMvZ/YUpAqA87A/hSo3JgdgC8=;EntityPath=eventhub'

In [9]:
os.environ.get('CONNECTION_STRING')

'Endpoint=sb://pythoneventhb.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=AQ/07wNguW0MtGKasw8IXcIoq9L3P44zwrFDsMLSWOg='

In [4]:
import asyncio
from azure.eventhub.aio import EventHubProducerClient
from azure.eventhub import EventData


    # Create a producer client to send messages to the event hub.
    # Specify a connection string to your event hubs namespace and
    # the event hub name.
producer = EventHubProducerClient.from_connection_string(conn_str="pythonEventHb-Endpoint=sb://pythoneventhb.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=AQ/07wNguW0MtGKasw8IXcIoq9L3P44zwrFDsMLSWOg=", eventhub_name="eventhub")
async with producer:
    # Create a batch.
    event_data_batch = await producer.create_batch()

    # Add events to the batch.
    event_data_batch.add(EventData('First event '))
    event_data_batch.add(EventData('Second event'))
    event_data_batch.add(EventData('Third event'))

    # Send the batch of events to the event hub.
    await producer.send_batch(event_data_batch)


ValueError: Connection string is either blank or malformed.